## <center><strong><span style= 'color: #51fcc6'>Notebook </span>Describtion

Hello There In this notebokk we are going to scrape the Full time and part time jobs data for the data related jobs like:<br>
Data analyst, ML dev, Data scientist, Data engineer etc ..<br><br>

**Important note:**
We will analyze some countries only for indeed becuase indeed needs to specify which country to look at.

## <center><strong>Importing <span style= 'color: #48e0dc'>Packeges</span>
<sub>*And setting constants

In [2]:
import re
import json
import time
import random
import scrapy
import pickle
import sqlite3
import warnings
import requests
import itertools
import numpy as np
import cloudscraper
import pandas as pd
import seaborn as sns
from fp.fp import FreeProxy
from itertools import count
from bs4 import BeautifulSoup
from selenium import webdriver
import matplotlib.pyplot as plt
from urllib.request import urlopen
from fake_useragent import UserAgent
from scrapy.http import HtmlResponse
from twisted.internet import reactor
from scrapy.crawler import CrawlerRunner
from scrapy.crawler import CrawlerProcess
from selenium.webdriver.common.by import By
from scrapy.utils.log import configure_logging
from selenium.webdriver.common.keys import Keys
from IPython.display import set_matplotlib_formats
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC

In [3]:
%matplotlib inline
COLORS = ['#51fcc6', '#48e0dc', '#5cd3f7', '#4895e0', '#517afc']

NUMERICS = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64',
            'uint16', 'uint32', 'uint64']
MAX_PAGES = 300

proxies = {'http': 'http://109.254.67.104:9090',
           'http': 'http://103.49.202.252:80'}

set_matplotlib_formats('pdf', 'svg')
warnings.filterwarnings('ignore')

C:\Users\FreeComp\AppData\Local\Temp\ipykernel_11508\1776924631.py:11: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('pdf', 'svg')


In [4]:
data_jobs_titles = ['Data entry', 'Data engineer',
                    'Data scientist', 'Data analyst',
                    'ML developer']

indeed_countries = ['de', 'uk']#, 'www'] USA don't have to be spesfied in the URL so we will use 'www' instead
linkedin_countries = ['European Union', 'United States']

## <center><strong>Setting up the<span style= 'color: #5cd3f7'> web scrapers

In [15]:
def scrape_page_fast(url: str) -> BeautifulSoup:

    scraper = cloudscraper.create_scraper(delay= 10,browser= {
        'browser': 'chrome',
        'platform': 'windows',
        'desktop': True,
        'mobile': False}) 
    
    content = scraper.get(url).text 
    return BeautifulSoup(content)



def scrape_page(url: str, retrieve_new_url= None, user_agent= None) -> BeautifulSoup:
    
    ua = UserAgent()
    
    # chrome_options = Options()
    # chrome_options.add_argument('--headless')  # Optional: Run Chrome in headless mode
    # chrome_options.add_argument(f"--user-agent={ua.random}")

    driver = webdriver.Chrome()
    driver.get(url)

    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))


    html = BeautifulSoup(driver.page_source)

    if retrieve_new_url:
        new_url = driver.current_url

        driver.quit()
        return html, new_url
        
    driver.quit()
    return html

In [6]:
# Comparing the two scrapers speed.
%timeit scrape_page_fast('https://linkedin.com')
%timeit scrape_page('https://linkedin.com')

# We can find that there's about 10X speed for the `scrape_page_fast` 🤯.

990 ms ± 98.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
9.18 s ± 618 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
# def indeed_scraper(page: int,
#                    country: str,
#                    job_title: str) -> BeautifulSoup:

#     modefid_page = (page - 1) * 10

#     data_job_title = job_title.replace(' ', '+')

#     # Pages in indeed.com are zero based indexed
#     url: str = f'https://{country}.indeed.com/jobs?q={data_job_title}&sc=0kf%3Aattr%28DSQF7%29%3B&start={modefid_page}' + \
#     '&pp=gQClAAAAAAAAAAAAAAACCXklWwCSAQIBK7YBBwDUbujiRGBahYW5TppjdUz7DjXn3aPZSbT47IoJ5LLbuzpYcXwZdzJ6rKHf' + \
#     '6gPWFkXVGxKKGxW-JAKb8BFo_hZAkBd7trBBTY32J2CrOuA3V9dGD_bre-lArmi9DRYlcah6hvoRfsYUNYSoQwIa8VOMZMxvH-s2Dlh' + \
#     'UPvUP-_Dz9ls4i-OLqVGpGh4AAA&vjk=b997bb0dddadea'

#     soup = scrape_page(url)
#     loaded_page = int(soup.find('button', 'css-ns2mzi e8ju0x51').text)

#     # available_pages = []

#     # for button in soup.find_all('button', 'css-1qt7hdn e8ju0x50'):
#     #     available_pages.append(int(button.text))

#     if (int(page) != loaded_page):
#         # We will use KeyError as standard to represent getting out of the max pages.
#         raise KeyError
        
#     print(f'Page {page} Loaded successfully.')

#     return  soup

In [8]:
def linkedin_scraper(country:   str, # This scraper scrapes all pages at once
                     job_title: str,
                     page:      int) -> BeautifulSoup:
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0',
        'Accept-Encoding': '*',
        'Connection': 'keep-alive'}

    page: int = (page - 1) * 25
    url: str = f'https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords={job_title}&' + \
    f'location={country}&geoId=90000084&trk=public_jobs_j%20obs-search-bar_search-submit&position=1&pageNum=0&start={page}'

    response = requests.get(url, headers= headers)
    soup = BeautifulSoup(response.content,'html.parser')

    return  soup

## <center><strong>Collecting<span style= 'color: #4895e0'> jobs </span> data

**What we will do in this section:**

1. Collect all pages we need from each scraper and stack them together as HTML code.
2. Scrape for jobs cards links from linkedin and indeed.
3. Scrape for the data in those links and store the data in a DataFrame
4. We may also consider collecting total results per each platform, country etc ..

#### **Stack pages together as HTML code**

In [9]:
def convert_to_integer(number_string):

    cleaned_string = ''.join(filter(str.isdigit, number_string))
    cleaned_string = cleaned_string.rstrip('+')
    integer_value = int(cleaned_string)

    return integer_value

In [10]:
stacked_df = {'soup':        [],
              'job_title':   [],
              'country':     [],
              'platform':    [],
              'total_jobs':  []}

flag     : bool = False
max_page : int = 4 # TOREMOVE

# Linked In
print('Starting LinkedIn scraper.')

for country in linkedin_countries:

    print(f'Loading the {country} data.')
    for job_title in data_jobs_titles:
        
        full_soup   : str = ''
        total_jobs  : list = convert_to_integer(scrape_page(f'https://www.linkedin.com/jobs/search?keywords={job_title}&location={country}&' + 
                                                            f'\geoId=&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0').find(
                                                                   'span', 'results-context-header__job-count').text)
        
        for i in count(0):
            page = i + 1
            page_soup: BeautifulSoup = linkedin_scraper(country= country,
                                                        job_title= job_title,
                                                        page= i + page)
            full_soup += str(page_soup)
            full_soup += ' <br> '

            if (page_soup.find_all('li') is None) or (page == max_page):
                break

        stacked_df['platform']      .append('LinkedIn')
        stacked_df['total_jobs']    .append(total_jobs) 
        stacked_df['soup']          .append(full_soup)
        stacked_df['job_title']     .append(job_title)
        stacked_df['country']       .append(country)

        print(f'Finished loading {job_title} jobs from the {country}.')
    print('\n')

Starting LinkedIn scraper.
Loading the European Union data.
Finished loading Data entry jobs from the European Union.
Finished loading Data engineer jobs from the European Union.
Finished loading Data scientist jobs from the European Union.
Finished loading Data analyst jobs from the European Union.
Finished loading ML developer jobs from the European Union.


Loading the United States data.
Finished loading Data entry jobs from the United States.
Finished loading Data engineer jobs from the United States.
Finished loading Data scientist jobs from the United States.
Finished loading Data analyst jobs from the United States.
Finished loading ML developer jobs from the United States.




In [11]:
# Indeed scraping TODO
# for country in indeed_countries:
#     for job_title in data_jobs_titles:
#         stacked_pages_soup: str = ''

#         for i in count(0):

#             try:
#                 page = i + 1
#                 soup: BeautifulSoup = indeed_scraper(country= country,
#                                                      job_title= job_title,
#                                                      page= page)

#                 stacked_pages_soup += '\n<br> ' + str(soup)
#                 if page > max_page:
#                     print('Finished loading current country or job title.\n')
#                     break

#             except KeyError as e:
#                 print('Finished loading current country or job title.\n')
#                 break

#         stacked_df['platform'].append('Indeed')
#         stacked_df['soup'].append(stacked_pages_soup)
#         stacked_df['job_title'].append(job_title)
#         stacked_df['country'].append(country)

#### **Collecting jobs & companies basic data**

In [12]:
def remove_escape_sequences(strings):

    new_strings = []
    escapes = ''.join([chr(char) for char in range(1, 32)])
    translator = str.maketrans('', '', escapes)

    for string in strings:
        new_string = string.translate(translator)
        new_strings.append(new_string)
        
    return tuple(new_strings)

In [13]:
stacked_df = pd.DataFrame(stacked_df)

stacked_df['jobs_locations']    = np.nan
stacked_df['listing_dates']     = np.nan
stacked_df['jobs_titles']       = np.nan
stacked_df['companies_names']   = np.nan

stacked_df['soups']   = np.nan
stacked_df['jobs_links']        = np.nan


# Collecting linkedIn jobs links
for i, soup in enumerate(stacked_df[stacked_df['platform'] == 'LinkedIn']['soup']):

    soup = BeautifulSoup(soup)

    linkedin_jobs_locations           = tuple([element.text for element in soup.find_all(
        'span', 'job-search-card__location')])

    linkedin_jobs_listing_dates       = tuple([element['datetime'] for element in soup.find_all(
        'time', 'job-search-card__listdate')])

    linkedin_jobs_titles              = tuple([element.text for element in soup.find_all(
        'h3', 'base-search-card__title')])

    linkedin_jobs_companies_names     = tuple([element.text for element in soup.find_all(
        'a', 'hidden-nested-link')])

    
    linkedin_jobs_links               = tuple([element['href'] for element in soup.find_all(
        'a', 'base-card__full-link absolute top-0 right-0 bottom-0 left-0 p-0 z-[2]')])
    
    linkedin_companies_links          = tuple([element['href'] for element in soup.find_all(
        'a', 'hidden-nested-link')])

    # I will use sets and tuples instead of lists because they are faster.
    stacked_df['jobs_locations']    [i] = remove_escape_sequences(linkedin_jobs_locations)
    stacked_df['listing_dates']     [i] = linkedin_jobs_listing_dates
    stacked_df['jobs_titles']       [i] = remove_escape_sequences(linkedin_jobs_titles)
    stacked_df['companies_names']   [i] = remove_escape_sequences(linkedin_jobs_companies_names)

    stacked_df['jobs_links']        [i] = linkedin_jobs_links
    stacked_df['soups']             [i] = linkedin_companies_links

In [13]:
# Collecting indeed jobs links TOFIX
# for i, soup in enumerate(stacked_df[stacked_df['platform'] == 'Indeed']['soup']):
    
#     indeed_jobs_links: list = []
#     indeed_companies_links: list = []

#     for job in BeautifulSoup(soup).find_all('a', 'jcs-JobTitle css-jspxzf eu4oa1w0'):
#         job_link = 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=###'.replace('###', job['data-jk'])
#         indeed_jobs_links.append(job_link)

#     for job_link in indeed_jobs_links[:5]: # WE WILL REMOVE THIS LIMIT
#         company_link = BeautifulSoup(scrape_page(job_link)).find('a', 'css-775knl emf9s7v0')
#         indeed_companies_links.append(company_link)
#         print(company_link)
        
#     stacked_df['total_jobs'][i] = len(indeed_jobs_links)
#     stacked_df['jobs_links'][i] = indeed_jobs_links
#     stacked_df['soups'][i] = indeed_companies_links

In [14]:

stacked_df.head()

,soup,job_title,country,platform,total_jobs,jobs_locations,listing_dates,jobs_titles,companies_names,companies_links,jobs_links
0,"\n<li>\n<div class=""base-card relative w-full ...",Data entry,European Union,LinkedIn,297000,"( Belmont, CA , ...","(2023-07-14, 2023-07-14, 2023-07-14, 2023-06-1...",( Data Entry Clerk (Typist)...,( Get.It Recruit - Administrative ...,(https://www.linkedin.com/company/get-it-recru...,(https://www.linkedin.com/jobs/view/data-entry...
1,"\n<li>\n<div class=""base-card relative w-full ...",Data engineer,European Union,LinkedIn,39000,"( San Ramon, CA , ...","(2023-06-15, 2023-04-07, 2023-04-28, 2023-05-2...",( Data Engineer ...,"( ShiftCode Analytics, Inc. ...",(https://www.linkedin.com/company/shiftcode-an...,(https://www.linkedin.com/jobs/view/data-engin...
2,"\n<li>\n<div class=""base-card relative w-full ...",Data scientist,European Union,LinkedIn,154000,"( San Francisco, CA , ...","(2023-06-16, 2023-07-10, 2023-07-06, 2023-05-0...",( Data Scientist ...,"( Experfy , Pa...",(https://www.linkedin.com/company/experfy?trk=...,(https://www.linkedin.com/jobs/view/data-scien...
3,"\n<li>\n<div class=""base-card relative w-full ...",Data analyst,European Union,LinkedIn,163000,"( San Francisco, CA , ...","(2023-02-20, 2023-05-11, 2023-05-08, 2023-01-2...",( Data Analyst ...,"( Digital Janet , ...",(https://www.linkedin.com/company/digitaljanet...,(https://www.linkedin.com/jobs/view/data-analy...
4,"\n<li>\n<div class=""base-card relative w-full ...",ML developer,European Union,LinkedIn,1000,"( San Francisco, CA , ...","(2023-06-05, 2023-06-29, 2023-07-05, 2023-05-1...",( Engineer (Full Stack / ML...,"( Sieve , Auro...",(https://www.linkedin.com/company/sievedata?tr...,(https://www.linkedin.com/jobs/view/engineer-f...


#### **Loading more jobs data**

In [15]:
# Collecting more LinkedIn data

# We will collect the: 
#    describtion, credential category, location type
#
stacked_df['describtion']         = None
stacked_df['credential_category'] = None
stacked_df['location_type']       = None
stacked_df['employment_type']     = None
stacked_df['industry']            = None
stacked_df['country']             = None

for i, jobs in enumerate([stacked_df['jobs_links'][0]]): # TOREMOVE remove the loop limit

    describtion:           list = []
    credential_category:   list = []
    location_type:         list = []
    employment_type:       list = []
    industry:              list = []
    country:               list = []
    
    for job in jobs:
            
        try:
            job_data:  dict = json.loads(scrape_page_fast(job).find(
                'script', attrs= {'type': 'application/ld+json'}).text)
        except:
            continue
            
        try: job_data['jobLocationType'] is True
        except: job_data['jobLocationType'] = np.nan
            
        try: job_data['applicantLocationRequirements'] is True
        except: job_data['applicantLocationRequirements'] = {'name': np.nan}

        try: job_data['educationRequirements'] is True
        except: job_data['educationRequirements'] = {'credentialCategory': np.nan}
        
        describtion               .append(job_data['description'].strip())
        credential_category       .append(job_data['educationRequirements']['credentialCategory'])
        location_type             .append(job_data['jobLocationType'])
        employment_type           .append(job_data['employmentType'].strip())
        industry                  .append(job_data['industry'].strip())
        country                   .append(job_data['applicantLocationRequirements']['name'])

    # Now we are going to convert list into tuples.
    stacked_df['describtion']          [i] = tuple(describtion)
    stacked_df['credential_category']  [i] = tuple(credential_category)
    stacked_df['location_type']        [i] = tuple(location_type)
    stacked_df['employment_type']      [i] = tuple(employment_type)
    stacked_df['industry']             [i] = tuple(industry)
    stacked_df['country']              [i] = tuple(country)

In [16]:
# TODO (loading indeed data.)

In [17]:
stacked_df.head()

,soup,job_title,country,platform,total_jobs,jobs_locations,listing_dates,jobs_titles,companies_names,companies_links,jobs_links,describtion,credential_category,location_type,employment_type,industry
0,"\n<li>\n<div class=""base-card relative w-full ...",Data entry,"(United States, United States, United States, ...",LinkedIn,297000,"( Belmont, CA , ...","(2023-07-14, 2023-07-14, 2023-07-14, 2023-06-1...",( Data Entry Clerk (Typist)...,( Get.It Recruit - Administrative ...,(https://www.linkedin.com/company/get-it-recru...,(https://www.linkedin.com/jobs/view/data-entry...,(Are you searching for a flexible and rewardin...,"(high school, high school, nan, bachelor degre...","(TELECOMMUTE, TELECOMMUTE, TELECOMMUTE, nan, T...","(FULL_TIME, FULL_TIME, FULL_TIME, PART_TIME, F...","(Human Resources Services, Human Resources Ser..."
1,"\n<li>\n<div class=""base-card relative w-full ...",Data engineer,None,LinkedIn,39000,"( San Ramon, CA , ...","(2023-06-15, 2023-04-07, 2023-04-28, 2023-05-2...",( Data Engineer ...,"( ShiftCode Analytics, Inc. ...",(https://www.linkedin.com/company/shiftcode-an...,(https://www.linkedin.com/jobs/view/data-engin...,None,None,None,None,None
2,"\n<li>\n<div class=""base-card relative w-full ...",Data scientist,None,LinkedIn,154000,"( San Francisco, CA , ...","(2023-06-16, 2023-07-10, 2023-07-06, 2023-05-0...",( Data Scientist ...,"( Experfy , Pa...",(https://www.linkedin.com/company/experfy?trk=...,(https://www.linkedin.com/jobs/view/data-scien...,None,None,None,None,None
3,"\n<li>\n<div class=""base-card relative w-full ...",Data analyst,None,LinkedIn,163000,"( San Francisco, CA , ...","(2023-02-20, 2023-05-11, 2023-05-08, 2023-01-2...",( Data Analyst ...,"( Digital Janet , ...",(https://www.linkedin.com/company/digitaljanet...,(https://www.linkedin.com/jobs/view/data-analy...,None,None,None,None,None
4,"\n<li>\n<div class=""base-card relative w-full ...",ML developer,None,LinkedIn,1000,"( San Francisco, CA , ...","(2023-06-05, 2023-06-29, 2023-07-05, 2023-05-1...",( Engineer (Full Stack / ML...,"( Sieve , Auro...",(https://www.linkedin.com/company/sievedata?tr...,(https://www.linkedin.com/jobs/view/engineer-f...,None,None,None,None,None


## <center><strong><span style= 'color: #517afc'>Saving</span> the data

In [27]:
jobs_df = pd.DataFrame()

for row in stacked_df.iterrows():
    print(row['cou

TypeError: tuple indices must be integers or slices, not str